<a href="https://colab.research.google.com/github/zhangchenbocalvin/city-search-tool/blob/main/city_search_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# imports
import pandas as pd
import numpy as np
import itertools
import random

In [53]:
peopele_ds = pd.read_csv('/content/data/people_ds.csv')

people= peopele_ds['Weights'].str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(',', '', regex=True).str.split(expand=True)


people.head(20)

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,2
3,0,0,0,0,0,3
4,0,0,0,0,1,0
5,0,0,0,0,1,1
6,0,0,0,0,1,2
7,0,0,0,0,1,3
8,0,0,0,0,2,0
9,0,0,0,0,2,1


In [54]:
from sklearn.metrics.pairwise import manhattan_distances

df = pd.DataFrame(np.random.randint(0, 2, (1, 6)))

df


,0,1,2,3,4,5
0,0,1,1,1,1,0


In [55]:
peopele_ds['md'] = pd.DataFrame(manhattan_distances(df, people)).T

In [64]:
topk_people=pd.DataFrame(peopele_ds.sort_values('md').head(20)['City'])

In [67]:
topk_people.sort_values('City')

,City
84,Aachen
405,Aachen
600,Bergen
597,Bergen
852,Bergen
341,Bergen
660,Canberra
404,Canberra
596,Canberra
324,Dresden
